In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Context_Aware_Description_Using_MiniLM_ViT_GPT2/


Mounted at /content/drive
/content/drive/My Drive/Context_Aware_Description_Using_MiniLM_ViT_GPT2


In [2]:
!pip install nltk
!pip install git+https://github.com/salaniz/pycocoevalcap


  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-1y9nu6nq
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-1y9nu6nq
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=06e01a6f21fa5d329b3ef640f9c4b49808300461ff7e06080d04d1f2ddb348be
  Stored in directory: /tmp/pip-ephem-wheel-cache-ub01nw7s/wheels/d2/1f/44/6485e566f8ae3d42b56e7c05fd50a3bbb70a50b0e6e7c55212
Successfully built pycocoevalcap


In [3]:
import os
import torch

from torch.utils.data import DataLoader
from transformers import get_scheduler
from torch.optim import AdamW

from tqdm import tqdm
from utils.flickr8k_dataset import Flickr8kDataset
from utils.model_training_validation import training_model

DATASET_PATH = 'Flickr8K'
caption_file_path = f'{DATASET_PATH}/captions.txt'
image_file_path = f'{DATASET_PATH}/images'


dataset = Flickr8kDataset(
    csv_path=caption_file_path,
    image_dir=image_file_path
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from sklearn.model_selection import train_test_split

# Split image-caption dictionary
train_data, val_data = train_test_split(dataset.data, test_size=0.1, random_state=42)

train_dataset = Flickr8kDataset(
    csv_path=caption_file_path,
    image_dir=image_file_path,
)
train_dataset.data = train_data

val_dataset = Flickr8kDataset(
    csv_path=caption_file_path,
    image_dir=image_file_path,
)
val_dataset.data = val_data

In [5]:
# from transformers import GPT2Tokenizer
# from sentence_transformers import SentenceTransformer

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

# minilm = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



# training_model(train_dataset, val_dataset, tokenizer)


Testing

In [7]:
from utils.flickr8k_dataset import Flickr8kDataset
from utils.model_training_validation import training_model, generate_caption
from utils.gpt2_prefix_caption_model import GPT2PrefixCaptionModel
from transformers import GPT2Tokenizer
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from PIL import Image

model = GPT2PrefixCaptionModel()
checkpoint_path = "./checkpoints/gpt2_caption_epoch10.pt"
model.load_state_dict(torch.load(checkpoint_path, map_location="cuda"))
model = model.cuda().eval()
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Visualize 5 samples
for i in range(5):
    sample = val_dataset[i]

    image_feat = sample["image_feat"].unsqueeze(0).cuda()
    context_embed = sample["context_embed"].unsqueeze(0).cuda()
    filename = sample["filename"]

    caption = generate_caption(model, image_feat, context_embed, tokenizer)

    # Load and show image
    img_path = os.path.join(image_file_path, filename)
    img = Image.open(img_path).convert("RGB")

    plt.imshow(img)
    plt.axis("off")
    plt.title("🖼 " + filename)
    plt.show()

    # Print captions
    print(f"📜 Generated: {caption}")
    print(f"✅ Ground Truths:")
    for ref in val_dataset.caption_dict[filename]:
        print(f" - {ref}")
    print("———")

Output hidden; open in https://colab.research.google.com to view.